In [1]:
import numpy as np
import pandas as pd

In [2]:
mapping_base_path = './data/'
city_path = mapping_base_path + 'city.en.txt'
region_path = mapping_base_path + 'region.en.txt'
tag_path = mapping_base_path + 'user.profile.tags.en.txt'
city_mapping = {str(x[0]):x[1] for x in np.array(pd.read_csv(city_path, header=None, sep='\t'))}
region_mapping = {str(x[0]):x[1] for x in np.array(pd.read_csv(region_path, header=None, sep='\t'))}
tag_mapping = {str(x[0]):x[1] for x in np.array(pd.read_csv(tag_path, header=None, sep='\t'))}

In [3]:
header = ['click','weekday','hour','timestamp','log_type',
          'user_id','user_agent','ip','region','city','ad_exchange',
          'domain','url','anonymous_url_id','ad_slot_id','ad_slot_width',
          'ad_slot_height','ad_slot_visibility','ad_slot_format','ad_slot_floor_price',
          'creative_id','key_page_url','adversiter_id','user_tags']
raw_data = pd.read_csv('./data/data_train.txt', header=None, sep='\t')

In [4]:
raw_data.columns = header

In [5]:
Y = np.array(raw_data['click'])
X = np.array(raw_data[header[1:len(header)]])
X_header = header[1:len(header)]
print X.shape
print Y.shape

(2847802, 23)
(2847802,)


In [6]:
def clean_by_line(line, header):
    rst = []
    non_use_keys = set(['anonymous_url_id'])
    for key,value in zip(header,line):
        value = str(value)
        key = str(key)
        if value == '' or key in non_use_keys:
            continue
        elif key == 'city':
            if city_mapping.get(str(value)) is not None:
                value = city_mapping.get(str(value))
                rst += [key+'_'+value]
        elif key == 'region':
            if region_mapping.get(str(value)) is not None:
                value = region_mapping.get(str(value))
                rst += [key+'_'+value]
        elif key == 'timestamp':
            rst += ['time_day_'+value[6:8]]
        elif key == 'user_agent':
            value = value.split('_')
            rst += ['user_agent_os_'+value[0]]
            rst += ['user_agent_browser_'+value[1]]
        elif key == 'user_tags':
            for sub_value in value.split(','):
                rst += ['user_tags_'+str(tag_mapping.get(str(sub_value)))]
        else:
            rst += [key+'_'+value]
    return rst

In [7]:
X = [clean_by_line(x,X_header) for x in X]
print X[0]

['weekday_6', 'hour_0', 'time_day_06', 'log_type_1', 'user_id_VhT3La5uDlaywOj', 'user_agent_os_windows', 'user_agent_browser_chrome', 'ip_122.233.40.*', 'region_zhejiang', 'city_hangzhou', 'ad_exchange_1', 'domain_trqRTummPvas1m58uG', 'url_762a1aa930e6f41c969e77226c62f3a7', 'ad_slot_id_mm_10058411_2865562_9929053', 'ad_slot_width_300', 'ad_slot_height_250', 'ad_slot_visibility_2', 'ad_slot_format_1', 'ad_slot_floor_price_0', 'creative_id_44966cc8da1ed40c95d59e863c8c75f0', 'key_page_url_361e128affece850342293213691a043', 'adversiter_id_3386', 'user_tags_Long-term interest/electronic game', 'user_tags_Long-term interest/entertainment', 'user_tags_Long-term interest/eduation', 'user_tags_Long-term interest/art&photography&design', 'user_tags_Long-term interest/online literature', 'user_tags_Demographic/gender/male']


In [8]:
import itertools
from collections import Counter
X_index = np.copy(X)
print len(X_index)

2847802


In [9]:
counts = Counter(itertools.chain(*X_index))

In [10]:
MIN_THRESHOLD = 3

In [11]:
# Mapping from index to word
filtered_feature_list = [x[0] for x in counts.most_common() if x[1] > MIN_THRESHOLD and x[1] < len(X_index)]
filtered_feature_set = set(filtered_feature_list)
# Mapping from word to index
inverse_index = {x: i for i, x in enumerate(filtered_feature_list)}

In [12]:
f_reverse_dict = open('./data/reverse_dictionary.csv','w')
for key in inverse_index.keys():
    f_reverse_dict.write(key+','+str(inverse_index.get(key))+'\n')
f_reverse_dict.close()

In [13]:
print len(Y),len(X)

2847802 2847802


In [14]:
f = open('./data/training.csv','w')
count = 0
for i in range(len(Y)):
    line = str(','.join([str(inverse_index.get(x)) for x in X[i] if inverse_index.get(x) != None]))
    f.write(line+'\n')
    count += 1
f.close()
print count

2847802
